# Image 11

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *  # Import the function
from pyspark.sql.functions import regexp_replace, col
from pyspark.sql.window import Window 
spark = SparkSession.builder.getOrCreate()


In [8]:
data = [
    ("C001", "2-24-01-01"),
    ("C001", "2-24-01-02"),
    ("C001", "2-24-01-03"),
    ("C001", "2-24-01-06"),
    ("C002", "2-24-01-03"), 
    ("C002", "2-24-01-04"),
    ("C003", "2-24-01-01"),
]

df = spark.createDataFrame(data, ["customer_id", "billing_date"])
df.show()
df = df.withColumn("billing_date", col("billing_date").cast(DateType()))

data_range_df = df.groupBy("customer_id").agg(
    min("billing_date").alias("start_date"),
    max("billing_date").alias("end_date")
).withColumn("date", explode(sequence("start_date", "end_date"))).drop("start_date", "end_date")


missing_df = data_range_df.join(df, (data_range_df["customer_id"] == df["customer_id"]) & (data_range_df["date"] == df["billing_date"]),"left_anti").withColumnRenamed("date", "missing_date")

missing_df.show()

{"ts": "2025-08-15 21:58:16.874", "level": "ERROR", "logger": "DataFrameQueryContextLogger", "msg": "[CAST_INVALID_INPUT] The value '2-24-01-01' of the type \"STRING\" cannot be cast to \"DATE\" because it is malformed. Correct the value as per the syntax, or change its target type. Use `try_cast` to tolerate malformed input and return NULL instead. SQLSTATE: 22018\n== DataFrame ==\n\"cast\" was called from\nline 13 in cell [8]\n\n\nJVM stacktrace:\norg.apache.spark.SparkDateTimeException\n\tat org.apache.spark.sql.errors.ExecutionErrors.invalidInputInCastToDatetimeErrorInternal(ExecutionErrors.scala:115)\n\tat org.apache.spark.sql.errors.ExecutionErrors.invalidInputInCastToDatetimeErrorInternal$(ExecutionErrors.scala:102)\n\tat org.apache.spark.sql.errors.ExecutionErrors$.invalidInputInCastToDatetimeErrorInternal(ExecutionErrors.scala:259)\n\tat org.apache.spark.sql.errors.ExecutionErrors.invalidInputInCastToDatetimeError(ExecutionErrors.scala:92)\n\tat org.apache.spark.sql.errors.Exe

+-----------+------------+
|customer_id|billing_date|
+-----------+------------+
|       C001|  2-24-01-01|
|       C001|  2-24-01-02|
|       C001|  2-24-01-03|
|       C001|  2-24-01-06|
|       C002|  2-24-01-03|
|       C002|  2-24-01-04|
|       C003|  2-24-01-01|
+-----------+------------+



DateTimeException: [CAST_INVALID_INPUT] The value '2-24-01-01' of the type "STRING" cannot be cast to "DATE" because it is malformed. Correct the value as per the syntax, or change its target type. Use `try_cast` to tolerate malformed input and return NULL instead. SQLSTATE: 22018
== DataFrame ==
"cast" was called from
line 13 in cell [8]


JVM stacktrace:
org.apache.spark.SparkDateTimeException
	at org.apache.spark.sql.errors.ExecutionErrors.invalidInputInCastToDatetimeErrorInternal(ExecutionErrors.scala:115)
	at org.apache.spark.sql.errors.ExecutionErrors.invalidInputInCastToDatetimeErrorInternal$(ExecutionErrors.scala:102)
	at org.apache.spark.sql.errors.ExecutionErrors$.invalidInputInCastToDatetimeErrorInternal(ExecutionErrors.scala:259)
	at org.apache.spark.sql.errors.ExecutionErrors.invalidInputInCastToDatetimeError(ExecutionErrors.scala:92)
	at org.apache.spark.sql.errors.ExecutionErrors.invalidInputInCastToDatetimeError$(ExecutionErrors.scala:88)
	at org.apache.spark.sql.errors